<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/LottoMax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [21]:
df = pd.read_csv("master.csv")

In [22]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  1025 non-null   object
 1   N01       1025 non-null   int64 
 2   N02       1025 non-null   int64 
 3   N03       1025 non-null   int64 
 4   N04       1025 non-null   int64 
 5   N05       1025 non-null   int64 
 6   N06       1025 non-null   int64 
 7   N07       1025 non-null   int64 
 8   N08       1025 non-null   int64 
dtypes: int64(8), object(1)
memory usage: 72.2+ KB


In [23]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [24]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 5
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [25]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [26]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 60

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(56,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(8))
# Adding the last output layer
model.add(Dense(number_of_features))

In [27]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=75)

# Train the model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=500, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/500
5/5 - 17s - loss: 0.9634 - accuracy: 0.1569 - 17s/epoch - 3s/step
Epoch 2/500
5/5 - 0s - loss: 0.9541 - accuracy: 0.2078 - 75ms/epoch - 15ms/step
Epoch 3/500
5/5 - 0s - loss: 0.9479 - accuracy: 0.2039 - 84ms/epoch - 17ms/step
Epoch 4/500
5/5 - 0s - loss: 0.9414 - accuracy: 0.1843 - 80ms/epoch - 16ms/step
Epoch 5/500
5/5 - 0s - loss: 0.9351 - accuracy: 0.1725 - 72ms/epoch - 14ms/step
Epoch 6/500
5/5 - 0s - loss: 0.9350 - accuracy: 0.1608 - 73ms/epoch - 15ms/step
Epoch 7/500
5/5 - 0s - loss: 0.9262 - accuracy: 0.1725 - 87ms/epoch - 17ms/step
Epoch 8/500
5/5 - 0s - loss: 0.9178 - accuracy: 0.2118 - 73ms/epoch - 15ms/step
Epoch 9/500
5/5 - 0s - loss: 0.9157 - accuracy: 0.2314 - 69ms/epoch - 14ms/step
Epoch 10/500
5/5 - 0s - loss: 0.9087 - accuracy: 0.2039 - 74ms/epoch - 15ms/step
Epoch 11/500
5/5 - 0s - loss: 0.9151 - accuracy: 0.2078 - 72ms/epoch - 14ms/step
Epoch 12/500
5/5 - 0s - loss: 0.8996 - accuracy: 0.2510 - 80ms/epoch - 16ms/step
Epoch 13/500
5/5 - 0s - loss: 0.8948 - 

In [28]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

32/32 [==============================] - 3s 7ms/step - loss: 1.1242 - accuracy: 0.3598
Test Loss: 1.1241713762283325, Test Accuracy: 0.3598039150238037
24/24 [==============================] - 0s 7ms/step - loss: 1.4981 - accuracy: 0.1582
Test Loss: 1.4980759620666504, Test Accuracy: 0.1581699401140213


In [29]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [30]:
''' import datetime
import pytz

# Get the current time in a specific timezone
tz = pytz.timezone('America/New_York')
current_day = datetime.datetime.now(tz)

time_label = "Midday" if (current_day.hour <= 14) else "Evening"

label = current_day.strftime("%Y_%m_%d_") + time_label
model.save(f'Models/{label}.keras')
print("saved model") '''

' import datetime\nimport pytz\n\n# Get the current time in a specific timezone\ntz = pytz.timezone(\'America/New_York\')\ncurrent_day = datetime.datetime.now(tz)\n\ntime_label = "Midday" if (current_day.hour <= 14) else "Evening"\n\nlabel = current_day.strftime("%Y_%m_%d_") + time_label\nmodel.save(f\'Models/{label}.keras\')\nprint("saved model") '

In [31]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict


<ipython-input-31-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08
1020,10,15,18,27,28,32,43,8
1021,5,13,27,28,32,45,50,44
1022,1,4,6,7,34,40,46,2
1023,1,6,9,11,31,39,50,35


In [32]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:5]

top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")
top_10_indices = np.argsort(y_pred[0], axis=0)[::-1][:10]
top_10_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_10_numbers.append(number)

print("The top 10 indices:", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers for the lottery game are:  ", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("The winning indices:", winning_indices)
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 7s 7s/step
The top 5 indices: [0 3 1 2 4]
The top 5 numbers: [2, 21, 6, 9, 22]


The top 10 indices: [0 3 1 2 4 5 6 7]
The top 10 numbers: [2, 21, 6, 9, 22, 32, 43, 3]


The predicted numbers for the lottery game are:   [ 3  7 14 17 27 29 32 36]
The actual numbers in the last lottery game were: [ 2  6  9 21 22 32 43  3]


The winning indices: [5 7]
winners: [ 3 32]
0.1


# Predict the next set of numbers

In [35]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 [==============================] - 0s 64ms/step
The predicted numbers for the lottery game: [ 3  7 14 17 27 29 32 36]


The top 5 indices [3 0 4 1 2]
The top 5 numbers: [24, 5, 27, 8, 13]


The top 10 indices [3 0 4 1 2 5 7 6]
The top 10 numbers: [24, 5, 27, 8, 13, 30, 4, 34]


winning indices [5 7]
winning numbers [30, 4]


In [34]:
next = df.copy()
next = next.tail(5)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 26ms/step
The top 5 indices [3 0 4 1 2]
The top 5 numbers: [24, 5, 27, 8, 13]


The top 10 indices [3 0 4 1 2 5 7 6]
The top 10 numbers: [24, 5, 27, 8, 13, 30, 4, 34]


winning indices [5 7]
winning numbers [30, 4]


The predicted numbers: [[ 5  8 13 24 27 30 34  4]]
